In [37]:
import json
import pandas as pd
import numpy as np

In [8]:
sentiment_data = []
for x in ['positive', 'negative', 'neutral']:
    with open('/home/echeng/llm-control/experiments/prompts/sentiment_prompts-10k/positive_prompts.jsonl', 'r') as f:
        l = [json.loads(line) for line in f]
    l = l[:100]
    for pt in l:
        pt['prompt'] = pt['prompt']['text']
        pt['continuation'] = pt['continuation']['text']
        pt['class'] = x
        sentiment_data.append(pt)
sentiment_df = pd.DataFrame(sentiment_data)

In [9]:
sentiment_df

,md5_hash,prompt,continuation,num_positive,class
0,c407590bd7adee91c20e54cb3e8d9407,"""It sticks true to",end of this series.,25,positive
1,071313d997ab550da23b46fa9b498f33,Loyal fans never miss,an important match.,25,positive
2,a8f9b1cca05dcd8036f6fb24a1f9f838,Confucianism is deeply rooted in,societies of east asia.,25,positive
3,62462719fff1c24d6e798403222fd3ca,A President unafraid to stand,for veterans and God!,25,positive
4,6ef1539c763591976580201fb2bb67c9,"""I am delighted to",have been working on this.,25,positive
...,...,...,...,...,...
295,476429b813700ecd2b5801f184a9f6c5,It is the full and true reality of our present,communion and participation in the One life of...,25,neutral
296,106e0c818282bcc215ad2d4202e1e442,Be the first to hear about Nation Travels dest...,", and explore the world with kindred spirits.",25,neutral
297,8efc4885a8e67d2fe30f2932d5bc326d,Being able to share and connect,with other people was pretty humbling.,25,neutral
298,36de9611420ddc6cb6e838ba53db72c4,Gill provides the Supreme Court with an opport...,a clear line on the constitutionality of parti...,25,neutral


In [10]:
sentiment_df.to_csv('/home/echeng/llm-control/experiments/test_sentiment.csv')

In [29]:
toxicity_df = []
with open('/home/echeng/llm-control/experiments/prompts/nontoxic_prompts-10k.jsonl', 'r') as f:
    l = [json.loads(line) for line in f]
    for pt in l:
        if 'toxicity' in pt['continuation'] and 'toxicity' in pt['prompt']:
            if pt['continuation']['toxicity'] is None or pt['prompt']['toxicity'] is None: continue
            pt['continuation_toxic'] = int(pt['continuation']['toxicity'] > 0.5)
            pt['prompt_toxic'] = int(pt['prompt']['toxicity'] > 0.5)
            pt['continuation'], pt['prompt'] = pt['continuation']['text'], pt['prompt']['text']
            
            toxicity_df.append(pt)
        else:
            continue
                                    

In [30]:
toxicity_df = pd.DataFrame(l).drop(columns=['challenging'])

In [41]:
toxic = toxicity_df[toxicity_df['continuation_toxic'] == 1].sample(150)
untoxic = toxicity_df[toxicity_df['continuation_toxic'] == 0].sample(150)
toxicity_df = pd.concat([toxic, untoxic])

In [43]:
toxicity_df.to_csv('/home/echeng/llm-control/experiments/test_toxicity.csv')

## Compile constraint set for sentiment

In [15]:
from datasets import load_dataset
from tqdm import tqdm
import pandas as pd

In [12]:
categories = ['raw_review_All_Beauty', 'raw_review_Toys_and_Games', 'raw_review_Cell_Phones_and_Accessories', 'raw_review_Industrial_and_Scientific', 'raw_review_Gift_Cards', 'raw_review_Musical_Instruments', 'raw_review_Electronics', 'raw_review_Handmade_Products', 'raw_review_Arts_Crafts_and_Sewing', 'raw_review_Baby_Products', 'raw_review_Health_and_Household', 'raw_review_Office_Products', 'raw_review_Digital_Music', 'raw_review_Grocery_and_Gourmet_Food', 'raw_review_Sports_and_Outdoors', 'raw_review_Home_and_Kitchen', 'raw_review_Subscription_Boxes', 'raw_review_Tools_and_Home_Improvement', 'raw_review_Pet_Supplies', 'raw_review_Video_Games', 'raw_review_Kindle_Store', 'raw_review_Clothing_Shoes_and_Jewelry', 'raw_review_Patio_Lawn_and_Garden', 'raw_review_Unknown', 'raw_review_Books', 'raw_review_Automotive', 'raw_review_CDs_and_Vinyl', 'raw_review_Beauty_and_Personal_Care', 'raw_review_Amazon_Fashion', 'raw_review_Magazine_Subscriptions', 'raw_review_Software', 'raw_review_Health_and_Personal_Care', 'raw_review_Appliances', 'raw_review_Movies_and_TV']

data = []
for category in categories:
    amazon = iter(load_dataset('McAuley-Lab/Amazon-Reviews-2023', category, streaming=True)['full'])
    
    for _ in tqdm(range(5000)):
        next_review = next(amazon)
        if next_review is not None:
            if next_review['rating'] != 3:
                data.append(next_review)

100%|██████████| 5000/5000 [00:00<00:00, 6145.80it/s]


In [16]:
data_df = pd.DataFrame(data)

In [18]:
data_df['negative'] = data_df['rating'] < 3
data_df = data_df[['negative', 'text']]

In [36]:
data_df = balance_labels('negative', data_df)

In [58]:
amazon = data_df

In [35]:
# Load and shuffle the dataset
def balance_labels(label_name, df):
    labels = df[label_name]
    minority = 1 if sum(labels) < 0.5 * len(labels) else 0
    
    # get all the minority labels
    minority_df = df[df[label_name] == minority]
    majority_df = df[df[label_name] == (1 if not minority else 0)].sample(len(minority_df))
    
    return pd.concat([minority_df, majority_df])

In [39]:
tweets = load_dataset('cardiffnlp/tweet_eval', 'sentiment')['train']

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/45615 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12284 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [41]:
tweets = pd.DataFrame(tweets)
tweets['negative'] = tweets['label'] == 0
tweets = tweets[['text', 'negative']]

In [43]:
tweets = balance_labels('negative', tweets)

In [44]:
tweets

,text,negative
7,So disappointed in wwe summerslam! I want to s...,True
13,That sucks if you have to take the SATs tomorrow,True
18,Amy Schumer sat down with The Hollywood Report...,True
42,@user how the hell does every one else get to ...,True
50,Thanks manager for putting me on the schedule ...,True
...,...,...
31610,"he was level ffs! also, he may be a problem ch...",False
1899,Wahhh I forgot the 2nd goal was so pretty in t...,False
23827,@user Similarity between #Mohandas Gandhi &amp...,False
6107,David Otunga says Brock Lesnar is the best ath...,False


In [45]:
yelp = load_dataset('Yelp/yelp_review_full')['train']

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [46]:
yelp = pd.DataFrame(yelp)
yelp.head()

,label,text
0,4,dr. goldberg offers everything i look for in a...
1,1,"Unfortunately, the frustration of being Dr. Go..."
2,3,Been going to Dr. Goldberg for over 10 years. ...
3,3,Got a letter in the mail last week that said D...
4,0,I don't know what Dr. Goldberg was like before...


In [47]:
yelp = yelp[yelp['label'] != 2]
yelp['negative'] = yelp['label'] < 2

In [49]:
yelp = yelp[['text', 'negative']]
yelp = balance_labels('negative', yelp)

In [50]:
yelp

,text,negative
0,dr. goldberg offers everything i look for in a...,False
2,Been going to Dr. Goldberg for over 10 years. ...,False
3,Got a letter in the mail last week that said D...,False
5,Top notch doctor in a top notch practice. Can'...,False
6,Dr. Eric Goldberg is a fantastic doctor who ha...,False
...,...,...
46716,Unless you plan on buying a $5000.00 bike or n...,True
75787,Let me now tell you the story of Taco Bell on ...,True
167030,"Such a 1 star place, but so cheap. With taxes...",True
246981,I'm NOT saying don't eat here. I AM saying mak...,True


In [51]:
imdb = load_dataset('stanfordnlp/imdb')['train']

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [53]:
imdb = pd.DataFrame(imdb)
imdb = imdb.rename(columns ={'plain_text': 'text'})

In [55]:
imdb['negative'] = imdb['label'] == 0

In [56]:
imdb = balance_labels('negative', imdb)

In [66]:
dfs = []

for df in imdb, yelp, tweets, amazon:
    dfs.append(df[~df['negative']].sample(3750))
    dfs.append(df[df['negative']].sample(3750))

sentiment_df = pd.concat(dfs)

In [67]:
sentiment_df = sentiment_df[['text', 'negative']]

In [70]:
shuffled_df = sentiment_df.sample(frac=1).reset_index(drop=True)

In [72]:
shuffled_df.to_csv('/home/echeng/llm-control/sentiment-constraint-set/train_shuffled_balanced.csv')

In [73]:
shuffled_df

,text,negative
0,This place has the best views in the valley fo...,False
1,First beware reviews by employees friends or o...,True
2,"Dieter Bohlen, Germany's notorious composer an...",True
3,This one is okay stocked but sometimes you nee...,True
4,"The first half was OK, but the last half reall...",True
...,...,...
29995,Okay. So there aren't really that many great m...,True
29996,This is a great size that hold quite a few pil...,False
29997,The service was terrible. The counter help cou...,True
29998,Khan says Chisora won\u2019t be the same after...,True
